In [9]:
import pandas as pd
import gc
import numpy as np

df = pd.read_parquet('./data/l_vm_completa_train.parquet', engine='fastparquet')
df = df[df['A_PREDECIR'] != 'N']
df = df.drop(columns=['A_PREDECIR'])
#df = pd.read_csv("../datasets/sell-in.txt.gz", sep="\t")


In [10]:

df = df.groupby(by=["PERIODO","PRODUCT_ID"]).agg({"TN":"sum"}).reset_index()
df["PERIODO"] = pd.to_datetime(df["PERIODO"], format="%Y%m")
df_pivot = df.pivot(index="PERIODO", columns="PRODUCT_ID", values="TN").reset_index()
df_pivot.columns.name = None



In [11]:
print(df_pivot.tail())

      PERIODO       20001       20002       20003       20004      20005  \
31 2019-08-01  1261.34529   813.78215   635.59563   482.13372  536.66800   
32 2019-09-01  1660.00561  1090.18771   967.77116   786.17140  879.52808   
33 2019-10-01  1561.50552  1979.53635  1081.36645  1064.69633  996.78275   
34 2019-11-01  1397.37231  1423.57739   948.29393   723.94206  606.91173   
35 2019-12-01  1504.68856  1087.30855   892.50129   637.90002  593.24443   

        20006      20007      20008      20009  ...    21248    21252  \
31  262.73593  307.82899  233.00983  520.41758  ...  0.02117  0.13446   
32  409.95501  369.74894  330.56343  558.45719  ...  0.01411  0.14427   
33  528.32630  445.34884  452.77197  556.15182  ...  0.01270  0.10025   
34  399.61420  357.85913  396.49833  711.89025  ...  0.02964  0.11982   
35  417.23228  390.43432  195.36854  495.03574  ...  0.01129  0.08560   

      21256    21259    21262    21263    21265    21266    21267    21276  
31  0.02965  0.03106  0.022

In [12]:

magicos = [ "PERIODO",20002, 20003, 20006, 20010, 20011, 20018, 20019, 20021,
   20026, 20028, 20035, 20039, 20042, 20044, 20045, 20046,  20049,
   20051, 20052, 20053, 20055, 20008, 20001, 20017, 20086, 20180,
   20193, 20320, 20532, 20612, 20637, 20807, 20838
 ]
X_train = df_pivot[magicos].query("PERIODO >= '2018-01-01' & PERIODO <= '2018-12-31' ")
X_train = X_train.T.iloc[1:]
X_train.columns = [f"t-{11-k}" for k in range(12)]


X_kgl = df_pivot.query("PERIODO >= '2019-01-01' & PERIODO <= '2019-12-31'")
X_kgl = X_kgl.T.iloc[1:]
X_kgl.columns = [f"t-{11-k}" for k in range(12)]
promedio = X_kgl.mean(axis=1)
promedio = promedio.fillna(0)


y = df_pivot[magicos].query("PERIODO == '2019-02-01'").T.iloc[1:]
y.columns = ["target"]


/tmp/ipykernel_48998/865624416.py:15: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  promedio = promedio.fillna(0)
/tmp/ipykernel_48998/865624416.py:18: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  y = df_pivot[magicos].query("PERIODO == '2019-02-01'").T.iloc[1:]


In [13]:
y

,target
20002,1043.01349
20003,758.32657
20006,479.99914
20010,337.76009
20011,431.62938
20018,265.84135
20019,323.66178
20021,219.88515
20026,263.67957
20028,277.49358


In [14]:
promedio

20001    1454.732720
20002    1175.437142
20003     784.976407
20004     627.215328
20005     668.270104
            ...     
21263       0.029993
21265       0.089541
21266       0.094659
21267       0.092835
21276       0.045447
Length: 780, dtype: float64

In [15]:
prod_menos12 = X_kgl.index[X_kgl.isna().sum(axis=1)> 0]
X_kgl = X_kgl[~X_kgl.index.isin(prod_menos12)]
promedio_menos12 = promedio[prod_menos12]

HIPERPARAMETROS

In [16]:
#predict 
productos_ok = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt", sep="\t")


In [32]:
display(X_train)

,t-11,t-10,t-9,t-8,t-7,t-6,t-5,t-4,t-3,t-2,t-1,t-0
20002,984.80167,712.00087,966.86044,999.20934,1103.39191,1033.82845,977.40239,1161.8843,954.23575,1378.49032,1766.81068,1009.45458
20003,907.56304,788.30749,778.55594,765.47838,784.35885,660.73323,656.227,955.97079,912.34156,1313.34211,1206.91773,769.82869
20006,470.33785,588.56272,862.19361,748.44391,865.28861,468.1526,515.20419,615.70617,478.04388,513.15472,566.66809,407.75925
20010,298.25586,477.48363,653.77607,171.74107,227.24082,223.87746,331.96807,582.83104,480.60235,612.50721,414.97753,285.02947
20011,340.75314,601.26066,526.99374,562.70214,484.04538,437.7555,300.26178,191.0727,189.5985,177.75576,289.13976,321.09714
20018,342.16945,337.54792,510.33171,543.06908,388.43687,420.33781,325.03223,375.61778,282.43485,161.58557,371.52958,326.01506
20019,243.71984,629.89543,321.26878,290.39581,356.42982,262.33076,351.0561,387.73155,417.95455,552.71975,532.98143,446.69747
20021,253.48383,234.06376,405.9533,94.17409,153.29935,447.59775,332.17776,259.45932,391.53891,404.8525,313.67485,228.25426
20026,250.81615,197.61244,203.14786,317.66739,378.08331,317.87268,330.87172,352.77241,397.8142,328.17556,307.11434,223.97681
20028,156.13416,184.6845,299.81952,186.19146,223.70166,220.42566,182.2275,254.6271,287.25606,272.74338,220.27824,225.29052


In [17]:
from sklearn.linear_model import LinearRegression

reg_model = LinearRegression()
reg_model.fit(X_train, y)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [18]:
reg_model.intercept_

array([0.44146718])

In [19]:
reg_model.coef_

array([[ 0.07367052,  0.11921107,  0.10380433,  0.14283912,  0.04393265,
         0.15193647, -0.00777452, -0.16187541, -0.0600306 ,  0.17820788,
         0.23655828, -0.00133878]])

In [20]:
X_kgl

,t-11,t-10,t-9,t-8,t-7,t-6,t-5,t-4,t-3,t-2,t-1,t-0
20001,1275.77351,1259.09363,1470.65653,1647.63848,1629.78233,1109.93769,1678.99318,1261.34529,1660.00561,1561.50552,1397.37231,1504.68856
20002,1266.78751,1043.01349,1083.62552,1287.62346,1034.98927,928.36431,1066.44999,813.78215,1090.18771,1979.53635,1423.57739,1087.30855
20003,964.76919,758.32657,638.0401,565.33774,590.12515,662.38654,715.20314,635.59563,967.77116,1081.36645,948.29393,892.50129
20004,511.33713,441.70332,619.77084,466.70901,603.31081,667.19411,521.71519,482.13372,786.1714,1064.69633,723.94206,637.90002
20005,363.58438,409.8995,488.21387,624.9988,897.26297,876.39696,745.74978,536.668,879.52808,996.78275,606.91173,593.24443
...,...,...,...,...,...,...,...,...,...,...,...,...
21248,0.02962,0.06209,0.05365,0.03106,0.01553,0.00988,0.02116,0.02117,0.01411,0.0127,0.02964,0.01129
21256,0.03811,0.05081,0.0593,0.01835,0.01553,0.00988,0.02822,0.02965,0.00423,0.00847,0.02682,0.01271
21259,0.04234,0.0508,0.06777,0.02117,0.01976,0.00988,0.04657,0.03106,0.00564,0.01975,0.02965,0.01412
21262,0.0268,0.05786,0.06353,0.01412,0.0113,0.0127,0.03953,0.02258,0.01552,0.01693,0.02682,0.01834


In [21]:
len(X_kgl.index)

656

In [ ]:
pred = pd.DataFrame({"product_id": X_kgl.index, "tn": reg_model.predict(X_kgl).flatten()})
nuevas_filas = []
for prod in productos_ok["product_id"]:
    if prod not in pred["product_id"].values:
        nuevas_filas.append({"product_id": prod, "tn": promedio[prod]})

pred = pd.concat([pred, pd.DataFrame(nuevas_filas)], ignore_index=True)
pred = pred[pred["product_id"].isin(productos_ok["product_id"])]


,product_id,tn
0,20001,1162.707525
1,20002,1183.640604
2,20003,684.763931
3,20004,580.484961
4,20005,563.560780
...,...,...
775,21252,0.178011
776,21265,0.089541
777,21266,0.094659
778,21267,0.092835


In [26]:
# Ordenar pred por product_id
pred = pred.sort_values(by="product_id").reset_index(drop=True)

In [27]:
display(pred.head(50))

,product_id,tn
0,20001,1162.707525
1,20002,1183.640604
2,20003,684.763931
3,20004,580.484961
4,20005,563.560780
5,20006,482.886867
6,20007,390.924420
7,20008,422.340199
8,20009,450.560618
9,20010,418.689888


In [30]:
# Renombrar columnas product_id como PRODUCT_ID y tn como TN
pred.columns = ["product_id", "tn"] 

In [33]:
pred.to_csv("reg_lineal5.csv", index=False)